In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Test access to gpu
torch.cuda.is_available()

True

In [3]:
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

Processing...
Done!


In [5]:
# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [6]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

model = LogisticRegression(input_size, num_classes)

In [7]:
# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [8]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
                   % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch: [1/5], Step: [100/600], Loss: 2.2233
Epoch: [1/5], Step: [200/600], Loss: 2.1270
Epoch: [1/5], Step: [300/600], Loss: 2.0427
Epoch: [1/5], Step: [400/600], Loss: 1.9621
Epoch: [1/5], Step: [500/600], Loss: 1.8386
Epoch: [1/5], Step: [600/600], Loss: 1.7430
Epoch: [2/5], Step: [100/600], Loss: 1.7237
Epoch: [2/5], Step: [200/600], Loss: 1.5665
Epoch: [2/5], Step: [300/600], Loss: 1.5880
Epoch: [2/5], Step: [400/600], Loss: 1.5346
Epoch: [2/5], Step: [500/600], Loss: 1.4679
Epoch: [2/5], Step: [600/600], Loss: 1.5281
Epoch: [3/5], Step: [100/600], Loss: 1.3796
Epoch: [3/5], Step: [200/600], Loss: 1.3834
Epoch: [3/5], Step: [300/600], Loss: 1.3547
Epoch: [3/5], Step: [400/600], Loss: 1.3376
Epoch: [3/5], Step: [500/600], Loss: 1.2710
Epoch: [3/5], Step: [600/600], Loss: 1.2180
Epoch: [4/5], Step: [100/600], Loss: 1.1931
Epoch: [4/5], Step: [200/600], Loss: 1.2212
Epoch: [4/5], Step: [300/600], Loss: 1.1720
Epoch: [4/5], Step: [400/600], Loss: 1.1619
Epoch: [4/5], Step: [500/600], L

In [9]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model
torch.save(model.state_dict(), 'model.pkl')

Accuracy of the model on the 10000 test images: 82 %
